# Healthcare on the Move: Patterns and Predictors of Medical Professional Migration in OECD Countries
**Elijah Adrian** (student id: 90293598)\
**Wuyang Ren** (student id: 87924817)\
**Pierre Lambelet** (student id: 46419362)\
**Angelo Mladineo** (student id: 61846507)

## Introduction

The Government of Canada states that health is an inalienable human right, crucial for sustainable development, economic prosperity, and poverty eradication, as well as being an indicator of a country's progress as health increases the likelihood for sustained growth ([Global Affairs Canada](https://www.international.gc.ca/world-monde/issues_development-enjeux_developpement/global_health-sante_mondiale/health_development-sante_developpement.aspx?lang=eng), 2017), and it is through healthcare systems where governments directly administer the health of a nation. Therefore, it is imperative to study healthcare systems, find patterns among their characteristics, and evaluate their performance constantly, to improve health outcomes for all citizens.

The purpose of this project is to analyze a key aspect of healthcare systems: the national background of healthcare workers. Such that we can check if there is a significant impact on the performance of healthcare systems because of the  background of its medical professionals. 
We will focus on OECD countries specifically, because they have a comprehensive database among them, from which patterns of migration and healthcare systems' performance can be observed.
For example, the latest OECD annual report on migration mentions that migration into OECD countries, in addition to employment of foreigners within them, has reached record high levels in 2022 ([OECD](https://www.oecd.org/migration/international-migration-outlook-1999124x.htm), 2023). 

Thus, in order to explore the impact of medical professionals backgrounds', this research project is divided into three points of interest with an additional section focused on maps showcasing how the migration of medical professionals looks like:

- The relationship between health outcomes and migration of healthcare providers 
- Government health insurance, healthcare spending, and remuneration of medical professionals as predictive factors of healthcare professional migration
- Relationship between domestic and foreign-trained medical professionals
- Maps

To explore these points of interest, we will use the OECD data to generate graphical representations and perform statistical analyses such that we can identify relationships in the data that indicate if foreign medical professionals are affecting healthcare systems significantly.

## Data

### Importing libraries

As it was mentioned in the introduction, the datasets in this project come from datasets found in [OECD data](https://data.oecd.org), but before we present and clean them, we start by importing the Python libraries that allow us to work with the data.

These are:
`math`, `pandas`, `numpy` , `geopandas`, `matplotlib.pyplot`, `seaborn`, and `statsmodels.api`.



In [ ]:
! pip install geopandas

In [ ]:
import math
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
pd.set_option('display.max_columns', None)
sns.set()

### Loading datasets

Now we read all the data files from the OECD. The naming convention we use is an abbreviation of the datasets' names (for example `demo_ref_raw` stands for **demo**graphic **ref**erence data and so on) and all of them have the suffix "raw" in their variable name to indicate that the datasets have not been processed.



In [ ]:
demo_ref_raw = pd.read_csv('demographicref.csv') #demographic reference data
econ_ref_raw = pd.read_csv('econref.csv') #economic reference data
health_expen_raw = pd.read_csv('healthexpenditure.csv') #healthcare expenditure data
health_qual_raw = pd.read_csv('healthquality.csv') #healthcare quality data
health_resor_raw = pd.read_csv('healthresources.csv') #healthcare resources data
health_util_raw = pd.read_csv('healthutil.csv') #healthcare utilization data
social_proc_raw = pd.read_csv('socialprotection.csv') #social healthcare protection data
worker_migr_raw = pd.read_csv('healthworkmigration.csv') #healthcare worker migration data

In [ ]:
health_stat_raw = pd.read_csv('healthstatus.csv',nrows = 284583) #health status data

Note that the final dataset (*healthstatus.csv*) cannot be loaded past the 284,614th line, so in order to avoid having partial data on one variable we needed to cut approximately the last 40,000 entries.

It is important to mention that all of these datasets were obtained from the [OECD database](https://data.oecd.org/).

To be more precise, each dataset we include was obtained from the following sources:
- [Demographic Reference Data](https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_DEMR)  
- [Economic Reference Data](https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_ECOR)  
- [Healthcare Expenditure Data](https://stats.oecd.org/Index.aspx?DataSetCode=SHA) 
- [Healthcare Quality Data](https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_HCQI)  
- [Healthcare Resources Data](https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_REAC)  
- [Healthcare Utilization Data](https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_PROC)  
- [Social Healthcare Protection Data](https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_PROT)  
- [Healthcare Worker Migration Data](https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_WFMI)  

Full citations are at the end of the document.

### Data cleaning

We start the data cleaning process by removing unwanted columns of data. We are left with columns that we deemed relevant for our project.

In [ ]:
demo_ref_col = demo_ref_raw[['Variable','Measure','Country','Year','Value']]
econ_ref_col = econ_ref_raw[['Variable', 'Measure', 'Country','Year', 'Value']]
health_expen_col = health_expen_raw[['Financing scheme', 'Function','Provider', 'Measure', 'Country', 'Year', 'Unit', 'Value']]
health_qual_col = health_qual_raw[['Country','Periods','Indicator','Gender','Age Group','Value','Value.1']]
health_resor_col = health_resor_raw[['Variable','Measure','Country','Year','Value']]
health_util_col = health_util_raw[['Variable','Measure','Country','Year','Value']]
social_proc_col = social_proc_raw[['Variable','Measure','Country','Year','Value']]
worker_migr_col = worker_migr_raw[['Country','Variable','Country of origin','Year','Value']]
health_stat_col = health_stat_raw[['Variable','Measure','Country','Year','Value']]

We proceed to remove unwanted rows of data. Typically, these are rows containing data measured in units that cannot be compared between countries (e.g. in local currency or raw numbers of incidents). In addition to that, we have to remove rows containing variables with few observations or rows that are unlikely to be helpful for our purposes. Almost all data sets required this, the exceptions are `demo_ref_col` and `worker_mig_col`.

In [ ]:
econ_ref_clean = econ_ref_col.loc[(econ_ref_col['Measure']=='National currency unit per US$, PPP')|
                       (econ_ref_col['Measure']=='/capita, US$ purchasing power parity')|
                       (econ_ref_col['Measure']=='Million US$ at exchange rate')|
                       (econ_ref_col['Measure']=='/capita, US$ exchange rate ')|
                       (econ_ref_col['Measure']=='/capita, US$ purchasing power parity')]

health_expen_clean = health_expen_col.loc[(health_expen_col['Measure']=='Share of gross domestic product')|
                       (health_expen_col['Measure']=='Constant prices, OECD base year')|
                       (health_expen_col['Measure']=='Share of current expenditure on health')|
                       (health_expen_col['Measure']=='Per capita, current prices, current PPPs')]

health_qual_clean = health_qual_col.loc[((health_qual_col['Value']=='Age-sex standardised rate per 100 000 population')|
               (health_qual_col['Value']=='Age-sex standardised rate per 100 patients')|
               (health_qual_col['Value']=='Age-standardised survival (%)'))|
               (health_qual_col['Value']=='Rate per 100 deaths (%)')|
               (health_qual_col['Value']=='Crude rate per 100 patients')|
               (health_qual_col['Value']=='Age-sex standardised ratio')|
               (health_qual_col['Value']=='Age-standardised survival (%)')|
               (health_qual_col['Value']=='Ratio')|
               (health_qual_col['Gender']=='Total')] 

health_qual_clean = health_qual_clean.drop(list((health_qual_clean.loc[(health_qual_clean['Periods']=="2000-2004")|
                                                                      (health_qual_clean['Periods']=="2005-2009")|
                                                                      (health_qual_clean['Periods']=="2010-2014")|
                                                                      (health_qual_clean['Periods']=="2015-2019")]).index.values)) 

health_resor_clean = health_resor_col.loc[(health_resor_col['Measure']=='Density per 1 000 population (head counts)')|
                (health_resor_col['Measure']=='Per million population')|
                (health_resor_col['Measure']=='% of total physicians (head counts)')|
                (health_resor_col['Measure']=='Per 1 000 population')|
                (health_resor_col['Measure']=='% of physicians (head counts)')|
                (health_resor_col['Measure']=='Per 100 000 population')|
                (health_resor_col['Measure']=='% of total hospital employment (head counts)')|
                (health_resor_col['Measure']=='Per 1 000 live births')|
                (health_resor_col['Measure']=='Salaried, income, US$ exchange rate')] 

health_util_clean = health_util_col.loc[(health_util_col['Measure']=='Per 100 000 population')|
               (health_util_col['Measure']=='% performed as inpatient cases')|
               (health_util_col['Measure']=='% performed as day cases')|
               (health_util_col['Measure']=='Per 100 000 females')|
               (health_util_col['Measure']=='Inpatient cases per 100 000 population ')|
               (health_util_col['Measure']=='Total procedures per 100 000 population')|
               (health_util_col['Measure']=='Day cases per 100 000 population')|
               (health_util_col['Measure']=='Per 1 000 population')|
               (health_util_col['Measure']=='% of children immunised')|
               (health_util_col['Measure']=='Waiting times from specialist assessment to treatment: Mean (days)')|
               (health_util_col['Measure']=='Waiting times of patients on the list: % of all patients waiting more than 3 months')|
               (health_util_col['Measure']=='Waiting times of patients on the list: Mean (days)')]

social_proc_clean = social_proc_col.loc[(social_proc_col['Measure']=='% of total population')|
                                        (social_proc_col['Measure']=='% of total population covered')] 

health_stat_clean = health_stat_col.loc[(health_stat_col['Measure']=='Deaths per 100 000 females (standardised rates)')|
               (health_stat_col['Measure']=='Deaths per 100 000 females (crude rates)')|
               (health_stat_col['Measure']=='Years lost, /100 000 females, aged 75 years old')|
               (health_stat_col['Measure']=='Deaths per 100 000 males (standardised rates)')|
               (health_stat_col['Measure']=='Deaths per 100 000 population (standardised rates)')|
               (health_stat_col['Measure']=='Deaths per 100 000 males (crude rates)')|
               (health_stat_col['Measure']=='Years lost, /100 000 males, aged 75 years old')|
               (health_stat_col['Measure']=='Years lost, /100 000 population, aged 75 years old')|
               (health_stat_col['Measure']=='Deaths per 100 000 population (crude rates)')|
               (health_stat_col['Measure']=='% of population (crude rate)')|
               (health_stat_col['Measure']=='Years')|
               (health_stat_col['Measure']=='Incidence per 100 000 population')|
               (health_stat_col['Measure']=='Deaths per 1 000 live births')]

Next we want to remove all the countries not present in the migration dataframe (`worker_migr_col`) from every other dataframe. We specifically select countries with a sufficient number of observations (we deemed 50 observations as appropriate). To do this we use a for loop that appends an empty list with countries if they have sufficient migration data and are not in the list already (to avoid for repeated entries). 

This results in the `migr_list` list, where we have all the countries (i.e. country names) that meet our observations criteria without duplicates.

In [ ]:
migr_list = []
for country in worker_migr_col["Country"]:
    if (worker_migr_col["Country"].value_counts()[country] > 50) and (country not in migr_list):
        migr_list.append(country)

We put `migr_list` into action immediately by filtering for all the rows in each dataframe where the *"Country"* column's value is in the list.

In [ ]:
econ_ref_clean = econ_ref_clean.loc[(econ_ref_clean["Country"].isin(migr_list))]
demo_ref_clean = demo_ref_col.loc[(demo_ref_col["Country"].isin(migr_list))]
health_expen_clean = health_expen_clean.loc[(health_expen_clean["Country"].isin(migr_list))]
health_qual_clean = health_qual_clean.loc[(health_qual_clean["Country"].isin(migr_list))]
health_resor_clean = health_resor_clean.loc[(health_resor_clean["Country"].isin(migr_list))]
health_util_clean = health_util_clean.loc[(health_util_clean["Country"].isin(migr_list))]
social_proc_clean = social_proc_clean.loc[(social_proc_clean["Country"].isin(migr_list))]
health_stat_clean = health_stat_clean.loc[(health_stat_clean["Country"].isin(migr_list))]

Lastly before pivoting we want to reset all the indexes.

In [ ]:
econ_ref_clean = econ_ref_clean.reset_index()
demo_ref_clean = demo_ref_clean.reset_index()
health_expen_clean = health_expen_clean.reset_index()
health_qual_clean = health_qual_clean.reset_index()
health_resor_clean = health_resor_clean.reset_index()
health_util_clean = health_util_clean.reset_index()
social_proc_clean = social_proc_clean.reset_index()
health_stat_clean = health_stat_clean.reset_index()

It is important to note that the dataframes in the code cell above have the suffix *"clean"*, this is to indicate that they have been processed (i.e. cleaned) and are ready for statistical analysis. We started using this naming convention before, and while these versions of the dataframes are not the ones we would present (a more suitable display is in the next code cell), it is these versions from where most of our work will stem. 

Next we have to pivot each dataframe, and merge the reference data, i.e. data that is not directly related to healthcare but that describes the countries we are going to use (such as purchasing power for `econ_ref` and employment for `demo_ref`).

In [ ]:
worker_migr = worker_migr_col.pivot_table(index = ['Year','Country'], columns = ['Variable'], values = 'Value')
demo_ref = demo_ref_clean.pivot_table(index = ['Year','Country'], columns = ['Measure','Variable'], values = 'Value')
econ_ref = econ_ref_clean.pivot_table(index = ['Year','Country'], columns = ['Measure','Variable'], values = 'Value')
health_expen = health_expen_clean.pivot_table(index = ['Year','Country'], columns = ['Measure','Function'], values = 'Value')
health_qual = health_qual_clean.pivot_table(index = ['Periods','Country','Age Group'], columns = ['Value','Indicator'], values = 'Value.1')
health_resor = health_resor_clean.pivot_table(index = ['Year','Country'], columns = ['Measure','Variable'], values = 'Value')
health_util = health_util_clean.pivot_table(index = ['Year','Country'], columns = ['Measure','Variable'], values = 'Value')
social_proc = social_proc_clean.pivot_table(index = ['Year','Country'], columns = ['Measure','Variable'], values = 'Value')
health_stat = health_stat_clean.pivot_table(index = ['Year','Country'], columns = ['Measure','Variable'], values = 'Value')

reference = pd.merge(demo_ref,econ_ref, on = ['Year','Country'])

In [ ]:
worker_migr

Here we have the final versions of our dataframes. To greatly simplify the information they present, we have eight dataframes that display data regarding healthcare, and one dataframe (the `reference` dataframe) that describes countries' economic and demographic conditions.

Most dataframes have the same hierarchical index, where we have 'Year' and 'Country' as the two indices. `healt_qual` is the exception, as it uses three indices, but all dataframes follow an initial index regarding time measurements, followed by a location ("Country") index.

## Relationship Between Health Outcome and Migration of Healthcare Providers 

To gauge the quality of acute care, we use the mortality and hospital readmission rate of the following conditions:
* Mortality within 30 days of admission
    * AMI(heart attack)
    * Haemorrhagic stroke
    * Ischemic stroke
*  Hip fracture surgery initiated within 2 calendar days after admission to the hospital 

We will also use questionnaire results, including the percentage of patients have enough consultation time and percentage of the population with perceived bad health conditions, as a measured of health status and patient satisfaction.

We filter data from the `health_qual_clean` dataframe and remove repeated entries.

In [ ]:
mor_val =  health_qual_clean["Value"] == "Age-sex standardised rate per 100 patients"
hos_val =  health_qual_clean["Value"] == "Age-sex standardised rate per 100 000 population" #Measure of mortality rate and hospital admission/readmission rate
doc_val = health_qual_clean["Value"] == "Crude rate per 100 patients"
doc_gender = health_qual_clean["Gender"] == "Total"

health_qual_clean['Periods'] = pd.to_numeric(health_qual_clean['Periods'])
yrs =  health_qual_clean["Periods"] > 2014

mor_ind = health_qual_clean["Indicator"].str.contains("mortality")
re_hos_ind = health_qual_clean["Indicator"].str.contains("hospital readmission")
doc_val_ind = health_qual_clean["Indicator"].str.contains("spending enough time with patients during the consultation")

qual_mor = health_qual_clean[mor_ind & yrs & mor_val]
qual_re_hos = health_qual_clean[re_hos_ind & yrs & hos_val]
qual_doc= health_qual_clean[doc_val_ind & yrs & doc_val & doc_gender ]


mor_countries = set(qual_mor['Country'].unique())
re_hos_countries = set(qual_re_hos['Country'].unique())
doc_countries = set(qual_doc['Country'].unique())

common_countries = mor_countries & re_hos_countries & doc_countries #filtering out countries

qual_mor_filtered = qual_mor[qual_mor['Country'].isin(common_countries)]
qual_re_hos_filtered = qual_re_hos[qual_re_hos['Country'].isin(common_countries)]
qual_doc_filtered = qual_doc[qual_doc['Country'].isin(common_countries)]

With our filtered data, we create new, small "summary" dataframes that allow for a clear plotting of average mortality and average hospital readmission rates since the year 2015.

In [ ]:
summary_qual_mor = pd.DataFrame(qual_mor_filtered.groupby(['Country'])['Value.1'].mean()).rename(columns={'Value.1': 'Mortality'})
summary_qual_re_hos = pd.DataFrame(qual_re_hos_filtered .groupby(['Country'])['Value.1'].mean()).rename(columns={'Value.1': 'Hospital Readmission'})
summary_qual_doc =  pd.DataFrame(qual_doc_filtered.groupby(['Country'])['Value.1'].mean()).rename(columns={'Value.1': 'Enough Consultation Time'})

We do the same to represent countries with "bad health", using the crude rate as a first filter (in this context, crude rate is a measurement of health quality spanning from "very good" to "very bad"), and then filtering specifically for "bad/very bad health" such that we can focus on the proportion of people that are perceived to have bad health conditions. 

In [ ]:
bad_health = pd.DataFrame(health_stat["% of population (crude rate)"]["Bad/very bad health, total aged 15+"]).rename(columns={'Bad/very bad health, total aged 15+': 'Bad Health'})
bad_health = bad_health[bad_health.index.get_level_values('Year') > 2014]
bad_health_filterd = bad_health[bad_health.index.get_level_values('Country').isin(common_countries)]

summary_bad_health = pd.DataFrame(bad_health_filterd.groupby(['Country'])['Bad Health'].mean()) 

We also need to consider the population age distribution of countries as it may be a factor that may influence, the perceived health of the a nation, and in turn how effective a healthcare system is. 

In [ ]:
age_ref = pd.DataFrame(reference["% of total population"]["Population: 65 years old and over"])
age_ref = age_ref[age_ref.index.get_level_values('Year') > 2014]
age_ref_filtered = age_ref[age_ref.index.get_level_values('Country').isin(common_countries)]

age_summary = pd.DataFrame(age_ref_filtered.groupby("Country").mean())

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax[0, 0] = summary_qual_mor.plot.bar(ax=ax[0,0])
ax[0, 0].set_title("Mortality Rate per 100 Patients")
ax[0, 1] = summary_qual_re_hos.plot.bar(ax=ax[0, 1])
ax[0, 1].set_title("Hospital Readmission Rate per 100 000 Population")
ax[1, 0] = summary_bad_health.plot.bar(ax=ax[1,0])
ax[1, 0].set_title("% of the Population with Perceived Bad Health Conditions")
ax[1, 1] = summary_qual_doc.plot.bar(ax = ax[1,1])
ax[1, 1].set_title("% of Patients Having Enough Consultation Time")
plt.tight_layout()

### Health Workforce Migration

In [ ]:
worker_migr_filtered = worker_migr[["Foreign-trained doctors - Annual inflow"]][worker_migr.index.get_level_values('Year') > 2014]
worker_migr_summary = worker_migr_filtered.groupby(['Country'])['Foreign-trained doctors - Annual inflow'].mean().fillna(0)
worker_migr_summary = worker_migr_summary[worker_migr_summary.index.get_level_values('Country').isin(common_countries)]
worker_migr_summary.plot.bar()

In [ ]:
per_capita_gdp = pd.DataFrame( reference["/capita, US$ purchasing power parity"]["Gross domestic product (GDP)"])
per_capita_gdp = per_capita_gdp[per_capita_gdp.index.get_level_values('Year') > 2014]
per_capita_gdp_filtered = per_capita_gdp[per_capita_gdp.index.get_level_values('Country').isin(common_countries)]

per_capita_gdp_summary = pd.DataFrame(per_capita_gdp_filtered.groupby(['Country'])['Gross domestic product (GDP)'].mean()).rename(columns={'Gross domestic product (GDP)': 'GDP per Capita by US$ purchasing power parity'})

In [ ]:
imm_qual = pd.merge(summary_qual_mor, summary_qual_re_hos, on = ['Country'])
imm_qual = pd.merge(imm_qual, summary_qual_doc, on = ['Country'])
imm_qual = pd.merge(imm_qual, summary_bad_health, on = ['Country'])
imm_qual = pd.merge(imm_qual, per_capita_gdp_summary, on = ['Country'])
imm_qual = pd.merge(imm_qual, worker_migr_summary, on = ['Country']) 
imm_qual = pd.merge(imm_qual, age_summary, on = ['Country']) 

In [ ]:
imm_correlation_matrix = imm_qual.corr()

In [ ]:
plt.figure(figsize=(7.5, 6))  
sns.heatmap(imm_correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=False)

plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.title('Correlation Between Health Outcomes and Healthcare Professionals Migration')

plt.show()

In [ ]:
imm_qual

In [ ]:
from sklearn import linear_model


X = imm_qual[["Mortality", "Enough Consultation Time", "Bad Health", "Hospital Readmission"]]
y = imm_qual['Foreign-trained doctors - Annual inflow'] 

X = sm.add_constant(X)
doc_imm_model = sm.OLS(y, X)
doc_imm_result = doc_imm_model.fit()

print(doc_imm_result.summary())

In [ ]:
lr_model = linear_model.LinearRegression()
lr_model.fit(X, y)
lr_coefs = pd.Series(dict(zip(list(X),lr_model.coef_)))
lasso_coefs_dict = {}
for alpha in [1, 5, 10, 15]:
    lasso_model = linear_model.Lasso(alpha)
    lasso_model.fit(X, y)
    lasso_coefs_dict[f'alpha = {alpha}'] = pd.Series(lasso_model.coef_, index = list(X))
    
coefs_comparison = pd.DataFrame(lasso_coefs_dict)
coefs_comparison

The coefficient do not shrink significantly with various levels of alpha in the Lasso model, indicating that all regressors have strong predictive power


### Results
In countries where the quality of acute healthcare is lower, as evidenced by higher mortality and hospital readmission rates, there is a tendency to have a higher proportion of immigrant doctors in the medical workforce. On the other hand, countries that report better patient satisfaction and overall health status are associated with a lower presence of immigrant doctors

## Insurance, Healthcare Spending, and Remuneration as Predictive Factors of Healthcare Professional Migration

This section investigates the predictive power and effects of government insurance, healthcare spending per capita, and healthcare professional remuneration on the migration of foreign healthcare professionals. While economic theory suggests a positive relationship between remuneration and migration, as healthcare workers seek better pay, it is unclear how the other factors will come into play. It is likely that high healthcare spending is related to high healthcare professional remuneration, so the relationships should be similar. As this is a purely predictive analysis, we are not concerned with multicollinearity. 

For this analysis it was necessary to remove the GP remuneration data due to a low number of observations, so we will use specialist remuneration as a proxy for overall physician salary. The relationship when plotted between healthcare spending and the dependent variables of interest suggested nonlinearity, so the logarithm was taken. The data on doctor and nurse inflow was highly skewed, but taking the log also resolved this. Since the data is over several years, we take the average of the data for each country to account for outlier years and prevent autocorrelation.

In [ ]:
log_hlth_percap_ppp = np.log(health_expen["Per capita, current prices, current PPPs"]["Current expenditure on health (all functions)"])
nur_sal_usd = health_resor["Salaried, income, US$ exchange rate"]["Remuneration of hospital nurses"]
spec_sal_usd = health_resor["Salaried, income, US$ exchange rate"]["Remuneration of specialists"]
gov_hi_pct = social_proc["% of total population"]["Government/compulsory health insurance coverage"]
for_doc_pct = worker_migr["% of foreign-trained doctors"]
for_nur_pct = worker_migr["% of foreign-trained nurses"]
log_for_doc_inf = np.log(worker_migr.loc[worker_migr["Foreign-trained doctors - Annual inflow"]!=0]["Foreign-trained doctors - Annual inflow"])
log_for_nur_inf = np.log(worker_migr.loc[worker_migr["Foreign-trained nurses - Annual inflow"]!=0]["Foreign-trained nurses - Annual inflow"])

In [ ]:
hlth_percap_ppp = health_expen["Per capita, current prices, current PPPs"]["Current expenditure on health (all functions)"]
nur_sal_usd = health_resor["Salaried, income, US$ exchange rate"]["Remuneration of hospital nurses"]
spec_sal_usd = health_resor["Salaried, income, US$ exchange rate"]["Remuneration of specialists"]
gov_hi_pct = social_proc["% of total population"]["Government/compulsory health insurance coverage"]
for_doc_pct = worker_migr["% of foreign-trained doctors"]
for_nur_pct = worker_migr["% of foreign-trained nurses"]
for_doc_inf = worker_migr["Foreign-trained doctors - Annual inflow"]
for_nur_inf = worker_migr["Foreign-trained nurses - Annual inflow"]

In [ ]:
in_hc_rem = pd.concat([hlth_percap_ppp,spec_sal_usd,nur_sal_usd,gov_hi_pct,for_doc_pct,for_doc_inf,for_nur_pct,for_nur_inf],axis=1)
in_hc_rem = in_hc_rem.groupby('Country').mean()
in_hc_rem["Current expenditure on health (all functions)"] = np.log(in_hc_rem["Current expenditure on health (all functions)"])
in_hc_rem["Foreign-trained doctors - Annual inflow"] = np.log(in_hc_rem["Foreign-trained doctors - Annual inflow"])
in_hc_rem["Foreign-trained nurses - Annual inflow"] = np.log(in_hc_rem["Foreign-trained nurses - Annual inflow"])
in_hc_rem = in_hc_rem.rename(columns = {"Current expenditure on health (all functions)":"log(Per capita current expenditure on health (PPP))",
                                       "Foreign-trained doctors - Annual inflow":"log(Foreign-trained doctors - Annual inflow)",
                                       "Foreign-trained nurses - Annual inflow":"log(Foreign-trained nurses - Annual inflow)"})

In [ ]:
doc_inf_pred = in_hc_rem[["log(Per capita current expenditure on health (PPP))","Remuneration of specialists",
                          "Government/compulsory health insurance coverage","log(Foreign-trained doctors - Annual inflow)"]].dropna()

doc_pct_pred = in_hc_rem[["log(Per capita current expenditure on health (PPP))","Remuneration of specialists",
                          "Government/compulsory health insurance coverage","% of foreign-trained doctors"]].dropna()

nur_inf_pred = in_hc_rem[["log(Per capita current expenditure on health (PPP))","Remuneration of hospital nurses",
                          "Government/compulsory health insurance coverage","log(Foreign-trained nurses - Annual inflow)"]].dropna()

nur_pct_pred = in_hc_rem[["log(Per capita current expenditure on health (PPP))","Remuneration of hospital nurses",
                          "Government/compulsory health insurance coverage","% of foreign-trained nurses"]].dropna()

### Correlations Between all Factors

In [ ]:
in_hc_rem_corr  = in_hc_rem.corr()

plt.figure(figsize=(7.5, 6))  
sns.heatmap(in_hc_rem_corr, annot=True, fmt=".2f", cmap="coolwarm", cbar=False)

plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.title('Correlation of foreign healthcare worker migration, healthcare worker remuneration, healthcare spending, and government insurance')

plt.show()

**Interpretation**
All factors are positively correlated with each other, except for government insurance coverage which is negatively correlated with most other factors. 

### Predictive Models

Predictive regressions of migration based on government insurance, healthcare spending, and remuneration.

In [ ]:
x_docinf = doc_inf_pred.loc[:,~doc_inf_pred.columns.isin(["log(Foreign-trained doctors - Annual inflow)"])]
y_docinf = doc_inf_pred["log(Foreign-trained doctors - Annual inflow)"]

x_docinf = sm.add_constant(x_docinf)
x_docinf_model = sm.OLS(y_docinf, x_docinf)
x_docinf_result = x_docinf_model.fit()

print(x_docinf_result.summary())

**Interpretation**

A 1% increase in per capita health spending gives a 1.65% increase in foreign doctor inflow. A $1000 increase in specialist remuneration gives a 1.78% increase in foreign doctor inflow. An increase in government healthcare coverage of 1 percentile point gives a 7.09% decrease in foreign doctor inflow.

In [ ]:
x_docpct = doc_pct_pred.loc[:,~doc_pct_pred.columns.isin(["% of foreign-trained doctors"])]
y_docpct = doc_pct_pred["% of foreign-trained doctors"]

x_docpct = sm.add_constant(x_docpct)
x_docpct_model = sm.OLS(y_docpct, x_docpct)
x_docpct_result = x_docpct_model.fit()

print(x_docpct_result.summary())

**Interpretation**

A 1% increase in per capita health spending gives a 0.13 percentile point increase in foreign doctor percentage. A $1000 increase in specialist remuneration gives a 0.10 percentile point increase in foreign doctor percentage. An increase in government healthcare coverage of 1 percentile point gives a 0.14 percentile point decrease in foreign doctor percentage.

In [ ]:
x_nurinf = nur_inf_pred.loc[:,~nur_inf_pred.columns.isin(["log(Foreign-trained nurses - Annual inflow)"])]
y_nurinf = nur_inf_pred["log(Foreign-trained nurses - Annual inflow)"]

x_nurinf = sm.add_constant(x_nurinf)
x_nurinf_model = sm.OLS(y_nurinf, x_nurinf)
x_nurinf_result = x_nurinf_model.fit()

print(x_nurinf_result.summary())

**Interpretation**

A 1% increase in per capita health spending gives a 0.78% increase in foreign nurse inflow. A $1000 increase in hospital nurse remuneration gives a 6.28% increase in foreign nurse inflow. An increase in government healthcare coverage of 1 percentile point gives a 0.66% decrease in foreign nurse inflow.

In [ ]:
x_nurpct = nur_pct_pred.loc[:,~nur_pct_pred.columns.isin(["% of foreign-trained nurses"])]
y_nurpct = nur_pct_pred["% of foreign-trained nurses"]

x_nurpct = sm.add_constant(x_nurpct)
x_nurpct_model = sm.OLS(y_nurpct, x_nurpct)
x_nurpct_result = x_nurpct_model.fit()

print(x_nurpct_result.summary())

**Interpretation**

A 1% increase in per capita health spending gives a 0.02 percentile point increase in foreign nurse percentage. A $1000 increase in hospital nurse remuneration gives a 0.20 percentile point increase in foreign nurse percentage. An increase in government healthcare coverage of 1 percentile point gives a 0.14 percentile point increase in foreign nurse percentage.

### Results
**Use interpretations**

## Relationship Between Domestic and Foreign-Trained Medical Professionals

In [ ]:
#create new column for percentage of domestically trained healthcare workers
#sum domestic trained doctors and nurses, divide by total
worker_migr['%_of_domestic_trained_healthcare_workers'] = (
    (worker_migr['Domestically-trained doctors'] + worker_migr['Domestically-trained nurses']) /
    (worker_migr['Total number of doctors'] + worker_migr['Total number of nurses'])
) * 100

worker_migr['%_of_foreign_trained_healthcare_workers'] = (
    (worker_migr['Foreign-trained doctors - Stock'] + worker_migr['Foreign-trained nurses - Stock']) /
    (worker_migr['Total number of doctors'] + worker_migr['Total number of nurses'])
) * 100


#worker_migr['%_of_workers_unknown_training'] = 100 - (worker_migr['%_of_foreign_trained_healthcare_workers']\
                                                    #+ worker_migr['%_of_domestic_trained_healthcare_workers'])


# Reset index
worker_migr= worker_migr.reset_index()

In [ ]:
worker_migr_to_plot = worker_migr[['Country', 'Year', '%_of_domestic_trained_healthcare_workers',
                                    '%_of_foreign_trained_healthcare_workers']] \
                        .groupby('Country') \
                        .mean() \
                        .dropna() \
                        .reset_index()

worker_migr_to_plot.head()

In [ ]:
bar_width = 0.4

# bar positions
r1 = np.arange(len(worker_migr_to_plot['Country']))
r2 = [x + bar_width for x in r1]

plt.figure(figsize=(12, 7.5))

#domestic healthcare workers
plt.bar(r1, worker_migr_to_plot['%_of_domestic_trained_healthcare_workers'], color='blue', width=bar_width, label='% of Domestic Trained Healthcare Workers')

#foreign healthcare workers
plt.bar(r2, worker_migr_to_plot['%_of_foreign_trained_healthcare_workers'], color='orange', width=bar_width, label='% of Foreign Trained Healthcare Workers')

plt.xticks([r + bar_width/2 for r in range(len(worker_migr_to_plot['Country']))], worker_migr_to_plot['Country'], rotation=90, size = 15)

plt.xlabel('Country', size = 15)
plt.ylabel('Percentage', size = 15)
plt.title('Percentage of Domestic and Foreign Trained Healthcare Workers by Country', size = 20)
plt.legend()
plt.tight_layout()
plt.show()

As seen in the figure above, most countries' healthcare workforce (doctors and nurses) is, understandably, predominantly local. However, some interesting patterns emerge. Ireland, Israel, New Zealand, Australia, and Switzerland stand out - over 20% of their healthcare workforce are foreign trained. While many factors affect these percentages, the OECD's 2015 Health Glance provides some insight; in Israel's case, many licenses are issued to doctors born domestically but who completed their training abroad. Although less prevalent, this pattern is also present in Norway, helping to explain their relatively high percentage of foreign trained health care workers. 

An interesting case study is Switzerland. Several factors appear to influence their healthcare workforce composition. Despite not being in the European Union, there is a free movement of people agreement between Switzerland and the EU. Switzerland's high quality of living and high salaries are appealing. This is evidenced by the large majority of these foreign-trained doctors being from neighboring countries, such as France and Italy(OFSP, 2023). Additionally, Switzerland has not trained enough doctors leading to a shortage of domestically-educated health care professionals.


## Maps

Maps of doctor inflow, foreign doctor percentage, nurse inflow, and foreign nurse percentage

In [ ]:
map_data = worker_migr[['Country', 'Year', '%_of_domestic_trained_healthcare_workers',
                        '%_of_foreign_trained_healthcare_workers', "% of foreign-trained doctors",
                        "% of foreign-trained nurses", "Foreign-trained doctors - Annual inflow",
                        "Foreign-trained nurses - Annual inflow"
                       ]] \
            .groupby('Country') \
            .mean() \
            .reset_index()

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

europe = world[world['continent'] == "Europe"]

europe = europe[europe['name'] != "Russia"]

#Replace 'United States of America' with 'United States' and 'Turkey' with 'Türkiye'
world['name'] = world['name'].replace({'United States of America': 'United States', 'Turkey': 'Türkiye'})


merged_data = world.merge(map_data, how='left', left_on='name', right_on='Country')

europe_data =merged_data[merged_data["continent"] == 'Europe']

europe_data = europe_data[europe_data['name'] != 'Russia']

In [ ]:
for variable in ['% of foreign-trained doctors', '% of foreign-trained nurses',
                 'Foreign-trained doctors - Annual inflow', 'Foreign-trained nurses - Annual inflow']:
    
    fig, ax = plt.subplots(2, 1, figsize=(10, 12))
    
    world.plot(ax=ax[0], color='white', edgecolor='black')
    ax[0].set_title(f'Map of {variable}')

    merged_data.plot(column=variable, cmap='OrRd', linewidth=0.8, ax=ax[0], edgecolor='0.8', legend=True)

    ax[0].text(0.15, 0.2, 'White areas indicate \nunavailable data', horizontalalignment='center', verticalalignment='center',
            transform=ax[0].transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
    
    europe.plot(ax=ax[1], color='white', edgecolor='black')
    europe_data.plot(column=variable, cmap='OrRd', linewidth=0.8, ax=ax[1], edgecolor='0.8', legend=True)
    ax[1].set_title(f'Map of {variable} (Zoomed in on Europe)')

    ax[1].text(0.15, 0.2, 'White areas indicate \nunavailable data', horizontalalignment='center', verticalalignment='center', 
            transform=ax[1].transAxes, fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
    
    plt.tight_layout()
    plt.show()


In [ ]:
from bokeh.io import output_notebook,show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool, Label, Legend
from bokeh.palettes import brewer
output_notebook()
import json
from bokeh.palettes import OrRd

In [ ]:
geo_source = GeoJSONDataSource(geojson=merged_data.to_json())

color_mapper = LinearColorMapper(palette=OrRd[9][::-1], low=0, high=45)

hover = HoverTool(tooltips=[('Country', '@name'),
                            ('% of foreign-trained doctors', '@{% of foreign-trained doctors}{0.2f}%'),
                            ('Annual inflow of foreign-trained doctors', '@{Foreign-trained doctors - Annual inflow}{0.2f}'),
                            ('% of foreign-trained nurses', '@{% of foreign-trained nurses}{0.2f}%'),
                            ('Annual inflow of foreign-trained nurses', '@{Foreign-trained nurses - Annual inflow}{0.2f}')])


p = figure(title="Interactive Map of OECD Healthcare Immigration", align = 'center', tools=[hover], width=1000, height=600)

p.title.align = 'center'

p.patches('xs', 'ys', source=geo_source,
          fill_color={'field': '% of foreign-trained doctors', 'transform': color_mapper},
          line_color='black', line_width=0.5)

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20,
                     border_line_color=None, location='bottom_center', orientation='horizontal', 
                     title="% of foreign-trained doctors")

footnote = Label(x=10, y=5, x_units='screen', y_units='screen',
                 text="Source: OECD (2023)", render_mode='css',
                 text_font_size='10pt', text_color='gray')

p.add_layout(footnote)


p.add_layout(color_bar, 'below')

output_notebook()
show(p)



## References
Global Affairs Canada. (2017). Global Health and Development. Retrieved from https://www.international.gc.ca/world-monde/issues_development-enjeux_developpement/global_health-sante_mondiale/health_development-sante_developpement.aspx?
lang=eng

OECD (2023), International Migration Outlook 2023, OECD Publishing, Paris, https://doi.org/10.1787/b0f40584-en

OECD (2023), *Demographic References*, OECD.Stat, https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_DEMR

OECD (2023), *Economic References*, OECD.Stat, https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_ECOR

OECD (2023), *Health Expenditure and Financing*, OECD.Stat, https://stats.oecd.org/Index.aspx?DataSetCode=SHA

OECD (2015), *International migration of doctors*, in Health at a Glance 2015: OECD Indicators, OECD
Publishing, Paris.

OECD (2023), *Healthcare Quality Indicators*, OECD.Stat, https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_HCQI

OECD (2023), *Healthcare Resources*, OECD.Stat, https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_REAC

OECD (2023), *Healthcare Utilization* OECD.Stat, https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_PROC

OECD (2023), *Social Protection*, OECD.Stat, https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_PROT

OECD (2023), *Health Workforce Migration*, OECD.Stat, https://stats.oecd.org/Index.aspx?DataSetCode=HEALTH_WFMI

OFSP(2023), *Statistiques médecins*, https://www.bag.admin.ch/bag/fr/home/zahlen-und-statistiken/statistiken-berufe-im-gesundheitswesen/statistiken-medizinalberufe1/statistiken-aerztinnen-aerzte.html
